## Interactive plots demo using NASA Exoplanet Archive

#### This tutorial will show you how to plot up an interactive exoplanet mass-radius diagram using bokeh or altair.

In [ ]:
import numpy as np

First we'll grab some data about known (confirmed) exoplanets from the [NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/index.html):

In [ ]:
from get_data import get_confirmed_planets
exoplanet_archive_table = get_confirmed_planets(select="*")

In [ ]:
exoplanet_archive_table.colnames

### Making an interactive mass-radius diagram with bokeh

Filter out all the planets with no known mass/radius, which are denoted with zeroes in the table:

In [ ]:
filter = (exoplanet_archive_table['pl_bmasse'] > 0.0) \
        & (exoplanet_archive_table['pl_rade'] > 0.0)

In [ ]:
print("There are {0} confirmed planets with measured mass + radius.".format(
    len(exoplanet_archive_table[filter])))

Now load up bokeh:

In [ ]:
from bokeh.plotting import ColumnDataSource, figure, show
from bokeh.io import output_notebook
output_notebook()

First we have to make the data source. Fortunately, this is easy from an astropy table -- all we have to do is make it a pandas dataframe with `.to_pandas()` and bokeh can handle the rest. We'll do that using the filtered table.

In [ ]:
table = exoplanet_archive_table[filter]
source = ColumnDataSource(table.to_pandas())

The first step is to set up the figure characteristics. We'll enable a few basic interactive functionalities here, as well as setting the axis ranges.

In [ ]:
fig = figure(tools="pan,wheel_zoom,box_zoom,reset", active_scroll="wheel_zoom",
            x_axis_type="linear", x_range=[0.0, 20.0], y_range=[0.0,5.0])

Now we can add features to the plot using the column names in the data source. To see what the figure looks like at any point, just do `show(fig)`.

In [ ]:
pl_render = fig.circle('pl_bmasse','pl_rade', source=source, size=10)
show(fig)

Not bad! We can scroll around and zoom in and out. But what if we want more functionality? Let's add some tooltips to tell us more about the planets on hover.

In [ ]:
from bokeh.models import HoverTool
hover = HoverTool(renderers=[pl_render],
                    tooltips=[
        ("name", "@pl_name"),
        ("mass", "@pl_bmasse{1.11} Earth masses"),
        ("radius", "@pl_rade{1.11} Earth masses"),
        ("discovered by", "@pl_discmethod")
        ]
    )
fig.add_tools(hover)

In [ ]:
show(fig)

One issue here is that we haven't included any uncertainty estimates. These are pretty important, especially once we get down to the smallest planets. 

Let's adjust the points so that opacity scales with the inverse variance. This will draw the viewer's eye to the best-measured planets.

In [ ]:
err_scale = (table['pl_masseerr1'] + table['pl_masseerr2'])**2/table['pl_bmasse']**2 \
                    + (table['pl_radeerr1'] + table['pl_radeerr2'])**2/table['pl_rade']**2 
err_weight = np.exp(-err_scale**0.15) # trial & error

In [ ]:
source.add(err_weight, name='err_weight')
pl_render.data_source = source # update source
pl_render.glyph.fill_alpha = 'err_weight'

We'll also render error bars and apply the same opacity to them. (There may be a more elegant way to do this, but this one works!)

In [ ]:
r_err_xs, r_err_ys = [], []
m_err_xs, m_err_ys = [], []
for pl in table:
    m = pl['pl_bmasse']
    r = pl['pl_rade']
    r_err_xs.append((m, m))
    r_err_ys.append((r + pl['pl_radeerr1'], r + pl['pl_radeerr2']))
    m_err_xs.append((m + pl['pl_masseerr1'], m + pl['pl_masseerr2']))
    m_err_ys.append((r, r))
fig.multi_line(r_err_xs, r_err_ys, line_alpha=err_weight)
fig.multi_line(m_err_xs, m_err_ys, line_alpha=err_weight)

We can also adjust the axis labels, toolbar location, etc:

In [ ]:
fig.xaxis.axis_label = 'Mass (Earth Masses)'
fig.yaxis.axis_label = 'Radius (Earth Radii)'
fig.xaxis.axis_label_text_font_size = '14pt'
fig.xaxis.major_label_text_font_size = '12pt'
fig.yaxis.axis_label_text_font_size = '14pt'   
fig.yaxis.major_label_text_font_size = '12pt' 
fig.toolbar_location = "above"

In [ ]:
show(fig)

Getting even fancier, let's make it so that clicking (or touch-tapping) takes you to a webpage for the planet:

In [ ]:
from bokeh.models import TapTool, OpenURL
fig.add_tools(TapTool())
url = "@pl_pelink"
taptool = fig.select(type=TapTool)
taptool.callback = OpenURL(url=url)

In [ ]:
show(fig)

If you want to save this plot to disk as an html file, you can do so like this:

In [ ]:
from bokeh.plotting import output_file, save
output_file('bokeh_massradius.html')
save(fig)

This html file is ready to host on the web. You can view a similar plot online at [bedell.space/dataviz/mr.html](http://bedell.space/dataviz/mr.html). Try viewing it on your phone or tablet to see how it works on mobile!

### Linking interactive plots with altair

Let's remake this basic plot in altair. It requires a bit less figure initialization to get started.

In [ ]:
import altair as alt
table.add_column(err_weight, name='err_weight')
data = table.to_pandas()

In [ ]:
alt.renderers.enable('notebook')

In [ ]:
points = alt.Chart(data).mark_point().encode(
    x='pl_bmasse:Q',
    y='pl_rade:Q',
    opacity='err_weight:Q'
).interactive()

In [ ]:
points

That was easy! Note that we did have to give altair [a data type](https://altair-viz.github.io/user_guide/encoding.html#data-types) in the form of the `:Q` (for quantitative, i.e. a non-discrete number), but otherwise it was able to infer information from the pandas dataframe pretty well and construct a nice plot with minimal setup.

In the above code, we were taking advantage of the built-in [shorthand](https://altair-viz.github.io/user_guide/encoding.html#encoding-shorthands) in altair. We can customize the appearance of the plot more by explicitly calling `alt.X`, `alt.Y`, etc., and changing the keyword arguments.

In [ ]:
points = alt.Chart(data).mark_point(clip=True, filled=True).encode(
    x=alt.X('pl_bmasse:Q', scale=alt.Scale(domain=(0, 20)),
           axis=alt.Axis(title='Mass (Earth Masses)')),
    y=alt.Y('pl_rade:Q', scale=alt.Scale(domain=(0, 5)),
           axis=alt.Axis(title='Radius (Earth Radii)')),
    opacity=alt.Opacity('err_weight:Q', legend=None)
).interactive()

In [ ]:
points

And let's add our error bars back in to finish it out. Here we're going to define some additional charts and [layer](https://altair-viz.github.io/user_guide/compound_charts.html#layer-chart) them on top using the `+` operator.

In [ ]:
data['mass_err1'] = data['pl_bmasse'] + data['pl_masseerr1']
data['mass_err2'] = data['pl_bmasse'] + data['pl_masseerr2']
data['rad_err1'] = data['pl_rade'] + data['pl_radeerr1']
data['rad_err2'] = data['pl_rade'] + data['pl_radeerr2']

In [ ]:
mass_error = alt.Chart(data).mark_rule().encode(
    x='mass_err1:Q',
    x2='mass_err2:Q',
    y='pl_rade:Q',
    color=alt.ColorValue('grey'),
    opacity=alt.Opacity('err_weight:Q', legend=None)
)

rad_error = alt.Chart(data).mark_rule().encode(
    x='pl_bmasse:Q',
    y='rad_err1:Q',
    y2='rad_err2:Q',
    color=alt.ColorValue('grey'),
    opacity=alt.Opacity('err_weight:Q', legend=None)
)

mass_error + rad_error + points

Now let's move on to my favorite part of altair: linking across charts! With this capability, we can define a selection brush that acts on any panel of a multi-panel plot, and the data points selected in one panel will be highlighted in the others.

To test this out, let's make an H-R diagram of the exoplanet host stars and link it to our mass-radius plot.

In [ ]:
data['st_abs_mag'] = data['gaia_gmag'] - 5.*(np.log10(data['gaia_dist']) - 1.)

In [ ]:
brush = alt.selection(type='interval', resolve='global')

planets = alt.Chart(data).mark_point(clip=True, filled=True, size=28).encode(
    x=alt.X('pl_bmasse:Q', scale=alt.Scale(domain=(0, 20)),
           axis=alt.Axis(title='Mass (Earth Masses)')),
    y=alt.Y('pl_rade:Q', scale=alt.Scale(domain=(0, 5)),
           axis=alt.Axis(title='Radius (Earth Radii)')),
    opacity=alt.Opacity('err_weight:Q', legend=None),
    color=alt.condition(brush, alt.ColorValue('blue'), alt.ColorValue('gray'))
).properties(
    selection=brush,
    width=300,
    height=300
)


stars = alt.Chart(data).mark_point(clip=True, filled=True, size=28).encode(
    x=alt.X('st_teff:Q', scale=alt.Scale(domain=(12e3, 3e3)),
           axis=alt.Axis(title='Effective Temperature (K)')),
    y=alt.Y('st_abs_mag:Q', scale=alt.Scale(domain=(15, -5)),
           axis=alt.Axis(title='Abs. G Magnitude')),
    opacity=alt.condition(brush, alt.OpacityValue(1.), alt.OpacityValue(0.4)),
    color=alt.condition(brush, alt.ColorValue('blue'), alt.ColorValue('gray'))
).properties(
    selection=brush,
    width=300,
    height=300
)

In [ ]:
planets | stars

(Note that we're relying on there being a good effective temperature and distance for all the stars, which isn't always true - one possible improvement would be to use the Gaia Bp-Rp colors and Bailer-Jones Gaia distances to make a color-magnitude diagram. If you're feeling ambitious, you can pull the data needed from [gaia-kepler.fun](http://gaia-kepler.fun) and have at it!)

It would be nice to pan around and also use the brush, but the keybindings (mousebindings?) conflict. We can actually change this with the "on" and "translate" keywords in the selection object. This isn't super obvious from the documentation, but if you open the chart in Vega Editor and poke around in the Javascript you can figure it out.

In [ ]:
brush = alt.selection(type='interval', resolve='global', 
                          on="[mousedown[event.shiftKey], window:mouseup] > \
                          window:mousemove!", zoom='False',
                          translate="[mousedown[event.shiftKey], window:mouseup] > \
                          window:mousemove!")

pan = alt.selection(type='interval', bind='scales',
                        on="[mousedown[!event.shiftKey], window:mouseup] > \
                        window:mousemove!",
                        translate="[mousedown[!event.shiftKey], window:mouseup] > \
                        window:mousemove!")

planets = alt.Chart(data).mark_point(clip=True, filled=True, size=28).encode(
    x=alt.X('pl_bmasse:Q', scale=alt.Scale(domain=(0, 20)),
           axis=alt.Axis(title='Mass (Earth Masses)')),
    y=alt.Y('pl_rade:Q', scale=alt.Scale(domain=(0, 5)),
           axis=alt.Axis(title='Radius (Earth Radii)')),
    opacity=alt.Opacity('err_weight:Q', legend=None),
    color=alt.condition(brush, alt.ColorValue('blue'), alt.ColorValue('gray'))
).properties(
    selection=brush+pan,
    width=300,
    height=300
)


stars = alt.Chart(data).mark_point(clip=True, filled=True, size=28).encode(
    x=alt.X('st_teff:Q', scale=alt.Scale(domain=(12e3, 3e3)),
           axis=alt.Axis(title='Effective Temperature (K)')),
    y=alt.Y('st_abs_mag:Q', scale=alt.Scale(domain=(15, -5)),
           axis=alt.Axis(title='Abs. G Magnitude')),
    opacity=alt.condition(brush, alt.OpacityValue(1.), alt.OpacityValue(0.4)),
    color=alt.condition(brush, alt.ColorValue('blue'), alt.ColorValue('gray'))
).properties(
    selection=brush+pan,
    width=300,
    height=300
)

In [ ]:
planets | stars

#### Other ideas for improvement/extension:

- implement [data linking in bokeh](https://bokeh.pydata.org/en/latest/docs/user_guide/interaction/linking.html)
- implement [tooltips in altair](https://altair-viz.github.io/gallery/multiline_tooltip.html)
- tidy up plot appearances in altair (particularly those error bars); investigate the [top-level chart configuration](https://altair-viz.github.io/user_guide/configuration.html) capabilities
- [link a histogram/bar chart with a scatter plot](https://altair-viz.github.io/gallery/interactive_cross_highlight.html) to highlight planet hosts in the color-magnitude diagram for different selections of planet radius bin, or different stellar metallicity bin